# 🎨 Data Designer 101: Structured Outputs and Jinja Expressions

[Click here](https://raw.githubusercontent.com/NVIDIA-NeMo/DataDesigner/refs/heads/main/docs/notebooks/2-structured-outputs-and-jinja-expressions.ipynb) to download this notebook to your computer.
#### 📚 What you'll learn

In this notebook, we will continue our exploration of Data Designer, demonstrating more advanced data generation using structured outputs and Jinja expressions.

If this is your first time using Data Designer, we recommend starting with the [first notebook](/notebooks/1-the-basics/) in this 101 series.

### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    CategorySamplerParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    ExpressionColumnConfig,
    InferenceParameters,
    LLMStructuredColumnConfig,
    ModelConfig,
    PersonFromFakerSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object that is used to interface with the library.


In [2]:
data_designer_client = DataDesigner()

[16:29:58] [INFO] ♻️ Using default model providers from '/Users/amanoel/.data-designer/model_providers.yaml'


### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](/models/default-model-settings.md) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

### 🧑‍🎨 Designing our data

- We will again create a product review dataset, but this time we will use structured outputs and Jinja expressions.

- Structured outputs let you specify the exact schema of the data you want to generate.

- Data Designer supports schemas specified using either json schema or Pydantic data models (recommended).

<br>

We'll define our structured outputs using [Pydantic](https://docs.pydantic.dev/latest/) data models

> 💡 **Why Pydantic?**
>
> - Pydantic models provide better IDE support and type validation.
>
> - They are more Pythonic than raw JSON schemas.
>
> - They integrate seamlessly with Data Designer's structured output system.


In [5]:
from decimal import Decimal
from typing import Literal

from pydantic import BaseModel, Field


# We define a Product schema so that the name, description, and price are generated
# in one go, with the types and constraints specified.
class Product(BaseModel):
    name: str = Field(description="The name of the product")
    description: str = Field(description="A description of the product")
    price: Decimal = Field(description="The price of the product", ge=10, le=1000, decimal_places=2)


class ProductReview(BaseModel):
    rating: int = Field(description="The rating of the product", ge=1, le=5)
    customer_mood: Literal["irritated", "mad", "happy", "neutral", "excited"] = Field(
        description="The mood of the customer"
    )
    review: str = Field(description="A review of the product")

Next, let's design our product review dataset using a few more tricks compared to the previous notebook.

In [6]:
# Since we often only want a few attributes from Person objects, we can
# set drop=True in the column config to drop the column from the final dataset.
config_builder.add_column(
    SamplerColumnConfig(
        name="customer",
        sampler_type=SamplerType.PERSON_FROM_FAKER,
        params=PersonFromFakerSamplerParams(),
        drop=False,
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home & Kitchen",
                "Books",
                "Home Office",
            ],
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_subcategory",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="product_category",
            values={
                "Electronics": [
                    "Smartphones",
                    "Laptops",
                    "Headphones",
                    "Cameras",
                    "Accessories",
                ],
                "Clothing": [
                    "Men's Clothing",
                    "Women's Clothing",
                    "Winter Coats",
                    "Activewear",
                    "Accessories",
                ],
                "Home & Kitchen": [
                    "Appliances",
                    "Cookware",
                    "Furniture",
                    "Decor",
                    "Organization",
                ],
                "Books": [
                    "Fiction",
                    "Non-Fiction",
                    "Self-Help",
                    "Textbooks",
                    "Classics",
                ],
                "Home Office": [
                    "Desks",
                    "Chairs",
                    "Storage",
                    "Office Supplies",
                    "Lighting",
                ],
            },
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="target_age_range",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=["18-25", "25-35", "35-50", "50-65", "65+"]),
    )
)

# Sampler columns support conditional params, which are used if the condition is met.
# In this example, we set the review style to rambling if the target age range is 18-25.
# Note conditional parameters are only supported for Sampler column types.
config_builder.add_column(
    SamplerColumnConfig(
        name="review_style",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1],
        ),
        conditional_params={
            "target_age_range == '18-25'": CategorySamplerParams(values=["rambling"]),
        },
    )
)

# Optionally validate that the columns are configured correctly.
config_builder.validate()

[16:29:58] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "customer",
        "product_category",
        "product_subcategory",
        "target_age_range",
        "review_style"
    ]
)

Next, we will use more advanced Jinja expressions to create new columns.

Jinja expressions let you:

- Access nested attributes: `{{ customer.first_name }}`

- Combine values: `{{ customer.first_name }} {{ customer.last_name }}`

- Use conditional logic: `{% if condition %}...{% endif %}`


In [7]:
# We can create new columns using Jinja expressions that reference
# existing columns, including attributes of nested objects.
config_builder.add_column(
    ExpressionColumnConfig(name="customer_name", expr="{{ customer.first_name }} {{ customer.last_name }}")
)

config_builder.add_column(ExpressionColumnConfig(name="customer_age", expr="{{ customer.age }}"))

config_builder.add_column(
    LLMStructuredColumnConfig(
        name="product",
        prompt=(
            "Create a product in the '{{ product_category }}' category, focusing on products  "
            "related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. The product should be priced between $10 and $1000."
        ),
        system_prompt=SYSTEM_PROMPT,
        output_format=Product,
        model_alias=MODEL_ALIAS,
    )
)

# We can even use if/else logic in our Jinja expressions to create more complex prompt patterns.
config_builder.add_column(
    LLMStructuredColumnConfig(
        name="customer_review",
        prompt=(
            "Your task is to write a review for the following product:\n\n"
            "Product Name: {{ product.name }}\n"
            "Product Description: {{ product.description }}\n"
            "Price: {{ product.price }}\n\n"
            "Imagine your name is {{ customer_name }} and you are from {{ customer.city }}, {{ customer.state }}. "
            "Write the review in a style that is '{{ review_style }}'."
            "{% if target_age_range == '18-25' %}"
            "Make sure the review is more informal and conversational."
            "{% else %}"
            "Make sure the review is more formal and structured."
            "{% endif %}"
        ),
        system_prompt=SYSTEM_PROMPT,
        output_format=ProductReview,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.validate()

[16:29:58] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "customer",
        "product_category",
        "product_subcategory",
        "target_age_range",
        "review_style"
    ]
    llm_structured_columns: ['product', 'customer_review']
    expression_columns: ['customer_name', 'customer_age']
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [8]:
preview = data_designer_client.preview(config_builder)

[16:29:58] [INFO] 🧐 Preview generation in progress
[16:29:58] [INFO] ✅ Validation passed
[16:29:58] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[16:29:58] [INFO] 🩺 Running health checks for models...
[16:29:58] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[16:30:00] [INFO]   |-- ✅ Passed!
[16:30:00] [INFO] 🎲 Preparing samplers to generate 10 records across 5 columns
[16:30:00] [INFO] 🧩 Generating column `customer_name` from expression
[16:30:00] [INFO] 🧩 Generating column `customer_age` from expression
[16:30:00] [INFO] 🗂️ Preparing llm-structured column generation
[16:30:00] [INFO]   |-- column name: 'product'
[16:30:00] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 1.0,
        "max_tokens": 1024,
        "max_parallel_requests": 4,
        "timeou

In [9]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ customer            │ {                                                                                         │
│                     │     'uuid': '02a9fa30-045c-4443-9223-d314546ce65b',                                       │
│                     │     'locale': 'en_US',                                                                    │
│                     │     'first_name': 'Steven',                                                               │
│                     │     'last_name': 'Caldwell',                                                              │
│                     │     'middle_name': None,                                                                  │
│                     │     'sex': 'Male',                                                                        │
│                     │     'street_number': '9869',                                                              │
│                     │     'street_name': 'Reynolds Alley',                                                      │
│                     │     'city': 'West Nicholas',                                                              │
│                     │     'state': 'Arizona',                                                                   │
│                     │     'postcode': '05872',                                                                  │
│                     │     'age': 71,                                                                            │
│                     │     'birth_date': '1954-01-09',                                                           │
│                     │     'country': 'Saint Vincent and the Grenadines',                                        │
│                     │     'marital_status': 'separated',                                                        │
│                     │     'education_level': 'some_college',                                                    │
│                     │     'unit': '',                                                                           │
│                     │     'occupation': 'Programmer, systems',                                                  │
│                     │     'phone_number': '001-301-970-1903x043',                                               │
│                     │     'bachelors_field': 'no_degree'                                                        │
│                     │ }                                                                                         │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_category    │ Home Office                                                                               │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_subcategory │ Storage                                                                                   │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ target_age_range    │ 35-50                                                                                     │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ review_style        │ brief                                                                                     │
├─────────────────────┼─────────────────────────────────

In [10]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,customer,product_category,product_subcategory,target_age_range,review_style,customer_name,customer_age,product,customer_review
0,{'uuid': '02a9fa30-045c-4443-9223-d314546ce65b...,Home Office,Storage,35-50,brief,Steven Caldwell,71,{'name': 'Ergonomic Under-Desk Storage Cabinet...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."
1,{'uuid': 'c2631f8f-5d17-4a41-b9f1-36297b9b9e8a...,Home Office,Storage,50-65,detailed,Robert May,61,{'name': 'Ergonomic Wall-Mounted File Cabinet'...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."
2,{'uuid': '5fb4e814-4186-4d55-a6ca-edef9c3df344...,Electronics,Cameras,50-65,structured with bullet points,Jennifer Morse,21,"{'name': 'RetroView Digital Camera', 'descript...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
3,{'uuid': '194eb1db-006e-45e8-ad48-48f9b4bf22ed...,Electronics,Headphones,65+,structured with bullet points,David Mays,24,"{'name': 'EarComfort Wireless Headphones', 'de...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
4,{'uuid': '727adea0-9197-46e2-a3ea-8b17585e9b6c...,Home Office,Lighting,50-65,rambling,Kerri Diaz,102,"{'name': 'Adjustable LED Desk Lamp', 'descript...","{'rating': 4, 'customer_mood': 'happy', 'revie..."
5,{'uuid': '5fc887bf-fdda-4698-8251-be51f9402a4a...,Clothing,Women's Clothing,35-50,structured with bullet points,Danielle Romero,21,"{'name': 'Elegant Floral Maxi Dress', 'descrip...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
6,{'uuid': '74c81e95-b43f-487d-b31e-7cf429fffbeb...,Electronics,Headphones,35-50,brief,Ryan Martinez,48,{'name': 'Premium Noise-Cancelling Over-Ear He...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."
7,{'uuid': '8bc6fdf7-1ba7-49cb-b06b-2d1e0f994a7e...,Home & Kitchen,Decor,50-65,rambling,Mary Weiss,59,"{'name': 'Timeless Heritage Wall Art Set', 'de...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
8,{'uuid': '94b5f377-8593-40af-81d2-cd5542b6ff33...,Home & Kitchen,Decor,35-50,detailed,Norma Meadows,90,"{'name': 'Modern Geometric Wall Art Set', 'des...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
9,{'uuid': '78b40c52-9f46-4486-a19a-bb9eca51f87d...,Home & Kitchen,Furniture,50-65,detailed,Brian Miller,29,{'name': 'Ergonomic Memory Foam Reading Chair'...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [11]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 9                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                    ┃       data type ┃            number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ customer                       │            dict │                     10 (100.0%) │          person_from_faker │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ product_category               │          string │                       4 (40.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ product_subcategory            │          string │                       7 (70.0%) │                subcategory │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ target_age_range               │          string │                       3 (30.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ review_style                   │          string │                       4 (40.0%) │                   category │
└────────────────────────────────┴─────────────────┴─────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                             🗂️ LLM-Structured Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product               │          dict │                10 (100.0%) │     268.0 +/- 0.7 │           77.0 +/- 8.7 │
├───────────────────────┼───────────────┼────────────────────────────┼───────────────────┼────────────────────────┤
│ customer_review       │          dict │                10 (100.0%) │     321.0 +/- 7.1 │        176.0 +/- 137.6 │
└───────────────────────┴───────────────┴───────────────

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [12]:
job_results = data_designer_client.create(config_builder, num_records=20)

[16:30:13] [INFO] 🎨 Creating Data Designer dataset
[16:30:13] [INFO] ✅ Validation passed
[16:30:13] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[16:30:13] [INFO] 🩺 Running health checks for models...
[16:30:13] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[16:30:14] [INFO]   |-- ✅ Passed!
[16:30:14] [INFO] ⏳ Processing batch 1 of 1
[16:30:14] [INFO] 🎲 Preparing samplers to generate 20 records across 5 columns
[16:30:14] [INFO] 🧩 Generating column `customer_name` from expression
[16:30:14] [INFO] 🧩 Generating column `customer_age` from expression
[16:30:14] [INFO] 🗂️ Preparing llm-structured column generation
[16:30:14] [INFO]   |-- column name: 'product'
[16:30:14] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 1.0,
        "max_tokens": 1024,
       

In [13]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,customer,product_category,product_subcategory,target_age_range,review_style,customer_name,customer_age,product,customer_review
0,"{'age': 93, 'bachelors_field': 'stem', 'birth_...",Books,Classics,35-50,structured with bullet points,Angela Mitchell,93,{'description': 'A curated selection of classi...,"{'customer_mood': 'happy', 'rating': 5, 'revie..."
1,"{'age': 45, 'bachelors_field': 'no_degree', 'b...",Clothing,Accessories,25-35,detailed,Alex Fletcher,45,{'description': 'A sleek and versatile crossbo...,"{'customer_mood': 'happy', 'rating': 5, 'revie..."
2,"{'age': 101, 'bachelors_field': 'arts_humaniti...",Home Office,Lighting,25-35,detailed,Kathryn Stafford,101,"{'description': 'A sleek, adjustable LED desk ...","{'customer_mood': 'happy', 'rating': 5, 'revie..."
3,"{'age': 42, 'bachelors_field': 'arts_humanitie...",Books,Non-Fiction,35-50,structured with bullet points,Daniel Freeman,42,{'description': 'A non-fiction book exploring ...,"{'customer_mood': 'happy', 'rating': 5, 'revie..."
4,"{'age': 39, 'bachelors_field': 'education', 'b...",Home Office,Desks,50-65,brief,Brittany Acosta,39,{'description': 'A sturdy and comfortable desk...,"{'customer_mood': 'happy', 'rating': 5, 'revie..."


In [14]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 9                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                    ┃       data type ┃            number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ customer                       │            dict │                     20 (100.0%) │          person_from_faker │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ product_category               │          string │                       5 (25.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ product_subcategory            │          string │                      11 (55.0%) │                subcategory │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ target_age_range               │          string │                       5 (25.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ review_style                   │          string │                       4 (20.0%) │                   category │
└────────────────────────────────┴─────────────────┴─────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                             🗂️ LLM-Structured Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product               │          dict │                20 (100.0%) │     268.0 +/- 0.8 │           63.5 +/- 7.5 │
├───────────────────────┼───────────────┼────────────────────────────┼───────────────────┼────────────────────────┤
│ customer_review       │          dict │                20 (100.0%) │     308.0 +/- 7.1 │         173.5 +/- 93.8 │
└───────────────────────┴───────────────┴───────────────

## ⏭️ Next Steps

Check out the following notebook to learn more about:

- [Seeding synthetic data generation with an external dataset](/notebooks/3-seeding-with-a-dataset/)
